In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from scipy import signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

import matplotlib.pyplot as plt
import scipy.io as sio

!pip install neurokit2
import neurokit2 as nk

import seaborn as sns

import pandas as pd
import numpy as np
import time
!pip install mne
import mne

     |████████████████████████████████| 983kB 2.8MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 6.6MB 2.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/DREAMERedit.mat"
raw = sio.loadmat(path)

In [ ]:
#Step 1 : frames
#from mne.time_frequency import psd_welch

def preprocess_EEG(raw, feature):

  frame = [[0 for x in range(640)] for y in range(12)] 

  #Dividing into frames
  for i in range(0,7680):
    j = i//640
    k = i%640
    frame[j][k] = raw[i]
  
  #frame[0][0-639] - Frame 1, frame[11][0-639] - Frame 12
  
  samplingRate = 128 
  nyquest = samplingRate/2.0
  n=213
  
  # delta 0-4 Hz Band Filter
  f = 4/nyquest
  f1 = 40/nyquest
  
  fdelta=[]
  pdelta=[]

  fnew = [[0 for x in range(640)] for y in range(18)]
  pnew = [[0 for x in range(640)] for y in range(18)]
  thetamean =[]
  alphamean =[]
  betamean  =[]
  gammamean =[]
  deltamean =[]

  delta_filter = signal.firwin(n, f, pass_zero=True, window ='hamming')
  overall_filter = signal.firwin(n, f1, pass_zero=True, window="hamming") #will be similar to orgiginal data

  for i in range(0,12):

    filt_overall = signal.filtfilt(overall_filter, 1, frame[i], method ='pad')
    filt_delta = signal.filtfilt(delta_filter, 1, filt_overall)
    f,p = signal.welch(filt_overall,fs =128, nperseg=256)
    
    fnew[i] = f
    pnew[i] = p
    
    #plt.semilogy(fnew[i], pnew[i])
    #plt.xlabel('frequency [Hz]')
    #plt.ylabel('PSD')

    # c has frequency and reading together
    c =[]
    for k in range(0,129):
      x=[pnew[i][k],fnew[i][k]]
      c.append(x)

    #method1 = mean 

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] <=4.0):  #delta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    deltamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >4.0 and c[a][1] <=7.5 ): #theta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    thetamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >7.5 and c[a][1]<=13):   #alpha
        count +=1
        sum += c[a][0] 
    mean = sum/count
    alphamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >13 and c[a][1]<=20):    #beta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    betamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >20 and c[a][1]<=40):    #gamma
        count +=1
        sum += c[a][0] 
    mean = sum/count
    gammamean.append(mean)
  
    fe=[]
    fe.append(deltamean[i])
    fe.append(thetamean[i])
    fe.append(alphamean[i])
    fe.append(betamean[i])
    fe.append(gammamean[i])
      
    feature.append(fe)

  return feature

In [ ]:
def feat_extract_EEG(raw, electrode):
    EEG_tmp = np.zeros((23, 18, 60))
    for participant in range(0, 23):
        for video in range(0, 18):

              B, S = [], []
                
              basl = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["baseline"][0, 0][video, 0][:, electrode])
              stim = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["stimuli"][0, 0][video, 0][:, electrode])
                
              B = preprocess_EEG(basl, B) # baseline psd received
              S = preprocess_EEG(stim, S) # stimulation psd received

              A = np.divide(S, B)   # relative density(absolute is diff for each individual)
              column = 0

              for k in range(0,5):
                for l in range(0,12):
                  EEG_tmp[participant, video,column] = A[l][k]
                  column+=1
                                

    col = []

    for i in range(0, 12):
      col.append("psddelta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdtheta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdalpha_"+str(i + 1)+"_un")
    for i in range(0, 12):
     col.append("psdbeta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdgamma_"+str(i + 1)+"_un")

    print(EEG_tmp.shape)
    EEG_tmp = EEG_tmp.reshape(-1,EEG_tmp.shape[2])
    print(EEG_tmp.shape)
    data_EEG = pd.DataFrame(EEG_tmp, columns=col)
    #scaler = StandardScaler()
    print(data_EEG)
    print(col)
    
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = data_EEG[[col[i]]]
        
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG

In [ ]:
def participant_affective(raw):
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            
            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            
            a[participant, video, 2] = int(participant+1)
            
            a[participant, video, 3] = int(video+1)
            
            a[participant, video, 4] = ["Searching for Bobby Fischer",
                                        "D.O.A.", "The Hangover", "The Ring",
                                        "300", "National Lampoon\'s VanWilder",
                                        "Wall-E", "Crash", "My Girl",
                                        "The Fly", "Pride and Prejudice",
                                        "Modern Times", "Remember the Titans",
                                        "Gentlemans Agreement", "Psycho",
                                        "The Bourne Identitiy",
                                        "The Shawshank Redemption",
                                        "The Departed"][video]
            
            a[participant, video, 5] = ["calmness", "surprise", "amusement",
                                        "fear", "excitement", "disgust",
                                        "happiness", "anger", "sadness",
                                        "disgust", "calmness", "amusement",
                                        "happiness", "anger", "fear",
                                        "excitement", "sadness",
                                        "surprise"][video]
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 7] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 8] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "video_name", "target_emotion",
                              "valence", "arousal", "dominance"])
    return b

In [ ]:
#RUN Each one by one till cell where is it savd as csv

#def_EEG1 = feat_extract_EEG(raw, 0)
#def_EEG2 = feat_extract_EEG(raw, 1)
def_EEG3 = feat_extract_EEG(raw, 2)
#def_EEG4 = feat_extract_EEG(raw, 3)
#def_EEG5 = feat_extract_EEG(raw, 4)
#def_EEG6 = feat_extract_EEG(raw, 5)
#def_EEG7 = feat_extract_EEG(raw, 6)
#def_EEG8 = feat_extract_EEG(raw, 7)
#def_EEG9 = feat_extract_EEG(raw, 8)
#def_EEG10 = feat_extract_EEG(raw, 9)
#def_EEG11 = feat_extract_EEG(raw, 10)
#def_EEG12 = feat_extract_EEG(raw, 11)
#def_EEG13 = feat_extract_EEG(raw, 12)
#def_EEG14 = feat_extract_EEG(raw, 13)

(23, 18, 60)
(414, 60)
     psddelta_1_un  psddelta_2_un  ...  psdgamma_11_un  psdgamma_12_un
0         0.026661       0.128126  ...        0.882784        0.788258
1         0.456918       0.854457  ...        0.982465        0.913652
2         0.249260       0.278371  ...        0.862490        0.793047
3         0.461071       0.972462  ...        0.893176        0.921639
4         0.612918       0.397175  ...        1.012029        1.154223
..             ...            ...  ...             ...             ...
409       0.138540       0.740260  ...        1.291717        0.776517
410       1.030576       0.510389  ...        0.771709        0.725069
411       0.485331       1.065134  ...        0.930742        1.022116
412       0.267938       1.659443  ...        0.743575        0.789763
413       0.780949       1.080066  ...        0.474525        0.581657

[414 rows x 60 columns]
['psddelta_1_un', 'psddelta_2_un', 'psddelta_3_un', 'psddelta_4_un', 'psddelta_5_un', 'psddelta_6_un

In [ ]:
#Enter desired electrode
df_features = def_EEG3

df_participant_affective = participant_affective(raw)

# not really clear to me why I have to do this because I thought these were already integers
df_participant_affective["valence"] = (df_participant_affective
                                       ["valence"].astype(int))
df_participant_affective["arousal"] = (df_participant_affective
                                       ["arousal"].astype(int))
df_participant_affective["dominance"] = (df_participant_affective
                                         ["dominance"].astype(int))

df = pd.concat([df_features, df_participant_affective], axis=1)
df2 = pd.concat([df_features, df_participant_affective], axis=1) ##
df

,psddelta_1,psddelta_2,psddelta_3,psddelta_4,psddelta_5,psddelta_6,psddelta_7,psddelta_8,psddelta_9,psddelta_10,psddelta_11,psddelta_12,psdtheta_1,psdtheta_2,psdtheta_3,psdtheta_4,psdtheta_5,psdtheta_6,psdtheta_7,psdtheta_8,psdtheta_9,psdtheta_10,psdtheta_11,psdtheta_12,psdalpha_1,psdalpha_2,psdalpha_3,psdalpha_4,psdalpha_5,psdalpha_6,psdalpha_7,psdalpha_8,psdalpha_9,psdalpha_10,psdalpha_11,psdalpha_12,psdbeta_1,psdbeta_2,psdbeta_3,psdbeta_4,psdbeta_5,psdbeta_6,psdbeta_7,psdbeta_8,psdbeta_9,psdbeta_10,psdbeta_11,psdbeta_12,psdgamma_1,psdgamma_2,psdgamma_3,psdgamma_4,psdgamma_5,psdgamma_6,psdgamma_7,psdgamma_8,psdgamma_9,psdgamma_10,psdgamma_11,psdgamma_12,age,gender,participant,video,video_name,target_emotion,valence,arousal,dominance
0,0.026661,0.128126,0.263066,0.205762,2.020957,0.382125,0.386174,0.393051,0.489584,0.697933,0.134432,0.248278,0.245683,0.540516,0.550646,0.554948,0.515048,0.767026,0.485870,0.663862,0.900809,0.714160,0.348549,0.528158,0.558594,0.750520,0.349834,0.942867,0.611311,0.879640,0.775826,0.881580,0.847456,1.301150,0.846869,0.613916,0.336742,0.680048,0.866783,1.402146,1.049143,0.796938,0.741151,1.296224,1.018292,1.565926,0.735186,0.765550,0.578959,0.819340,0.739843,0.820724,0.682887,0.872524,0.848397,0.767168,0.750893,0.875274,0.882784,0.788258,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,0.456918,0.854457,1.389768,0.418616,0.444282,1.150945,1.379311,0.951290,0.990952,0.587898,0.526920,0.597184,0.541314,0.613755,0.607566,0.956216,0.255011,0.274969,0.257066,0.411898,0.334922,0.414285,1.015785,0.683988,0.626123,0.660259,1.511112,0.823077,0.456709,0.415401,0.860516,0.402102,0.950805,1.119504,0.381495,0.398225,0.832206,0.519201,0.531851,0.800187,0.745460,0.676006,0.714218,0.768573,0.751424,0.531997,1.120774,0.819686,1.026307,0.810284,0.974069,0.798344,0.839761,0.887248,1.083175,0.968977,0.874396,0.855578,0.982465,0.913652,22,male,1,2,D.O.A.,surprise,3,3,1
2,0.249260,0.278371,0.645247,0.561594,0.405477,1.393356,0.415535,61.628447,54.163474,0.894641,0.655756,0.577133,0.021824,0.933490,0.299910,0.710720,0.936751,0.672708,0.825130,50.162477,54.494428,0.761028,0.958375,1.421353,0.098920,0.559696,0.529725,0.697565,0.550774,0.885256,0.586864,49.798175,97.586580,0.722648,0.908019,0.654330,0.407342,0.540492,0.980841,0.504885,0.598663,0.427006,1.319814,8.104653,111.160500,0.708537,0.672024,1.136864,0.782554,1.020950,0.966922,0.727917,0.945849,0.722588,0.686958,1.280376,2.198474,0.802747,0.862490,0.793047,22,male,1,3,The Hangover,amusement,5,4,4
3,0.461071,0.972462,0.878281,0.657676,1.148692,0.881123,0.523188,0.636787,1.049582,0.879103,1.341587,1.477064,0.181619,0.798121,0.534931,0.386719,1.022062,0.870922,0.667982,0.716668,0.401747,0.679756,0.726421,0.755361,0.574776,0.493555,0.899483,0.821588,0.951741,0.861824,0.792229,1.448838,1.073606,1.218513,1.188021,1.169386,0.598369,0.757336,1.161237,0.479530,0.563567,0.751573,0.908230,0.649187,0.758692,0.929840,0.641589,0.900956,0.996206,0.761949,0.995205,0.790317,0.754718,0.876863,0.836408,1.001039,0.892834,0.883298,0.893176,0.921639,22,male,1,4,The Ring,fear,4,3,2
4,0.612918,0.397175,0.419856,0.409736,0.800813,157.594122,7.629884,2.357575,1.201276,0.801799,0.990044,0.097768,0.904371,0.833003,0.558178,0.529204,0.493676,3.089034,0.773680,0.948897,1.356528,1.986031,1.060070,0.074344,0.902573,0.462086,0.912282,0.787960,1.126032,1.995489,0.870255,0.616019,0.568001,0.573661,0.539903,0.103279,0.845877,0.531877,0.978822,0.810316,0.657459,0.805898,0.643490,0.952502,0.998672,0.792066,0.471923,0.306508,0.934004,0.979796,1.083026,0.945325,1.125120,1.462209,1.538712,0.982351,1.034477,0.944886,1.012029,1.154223,22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.138540,0.740260,0.137076,0.722844,0.729615,2.028608,1.

In [ ]:
data2 = df2.loc[(df['target_emotion'] == 'anger') |
                (df['target_emotion'] == 'fear') |
                (df['target_emotion'] == 'calmness') |
                (df['target_emotion'] == 'surprise') |
                (df['target_emotion'] == 'excitement') |
                (df['target_emotion'] == 'amusement') |
                (df['target_emotion'] == 'happiness') |
                (df['target_emotion'] == 'sadness') |
                (df['target_emotion'] == 'disgust')].copy()

d={'surprise': 0, 'excitement': 0, 'amusement': 0, 'happiness': 0, 'fear': 1, 'anger': 1, 'sadness':2, 'disgust':2 ,'calmness': 3}
data2['class'] = data2.target_emotion.map(d)

e={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['valencehigh'] = data2.valence.map(e)

f={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['arousalhigh'] = data2.arousal.map(e)

# add another column with title 'stress_bin' and give it value 0/1 depending on what emotion it elicits
#data['stress_bin'] = data['target_emotion'].map({'anger': 1, 'fear': 1, 'calmness': 0})

# save features, demographic, and emotion data as csv
data2.to_csv('DE3_NS.csv')

In [ ]:
data2 = pd.read_csv("ml_data_untouched.csv")  
data_2 = data2.head()
data_2  

NameError: ignored

In [ ]:
## SOME CHECKS
# print( type(data))
#data = pd.read_csv("ml_data.csv")  
#data_ = data.head()  


In [ ]:
#DATA PREP FOR VALENCE

#data  - ml_data.csv
#data2 - ml_data_untouched.csv
#Valence is judged by alphas of F3 and F4
#F3 is data 3, F4 is data 12

#df1=data2.reset_index()['psdalpha_12']
#df2=data2.reset_index()['psdalpha_3']
#df1 = np.array(df1, dtype = float)
#df2 = np.array(df2, dtype = float)
#df3 = np.concatenate((df1, df2), axis=0)
#data2=data2.reset_index()
dfz=data2.iloc[:,1:43]
#dfy=pd.DataFrame(data={ 'alpha12':df1,'alpha3':df2})
#df1 = df1.append(df2)
dfz = np.array(dfz, dtype = float)
dfz

In [ ]:
target=data2['valencehigh']
#target = target.append(target)
target

In [ ]:
from sklearn.model_selection import train_test_split
#df1 = np.array(df1, dtype = float)
#df2 = np.array(df2, dtype = float)
target = np.array(target, dtype = float)


x_Train, x_Test, y_Train, y_Test = train_test_split(dfz, target, test_size = 0.35, random_state = 6)

In [ ]:
x_Train = x_Train.reshape(x_Train.shape[0],1,42)
x_Test = x_Test.reshape(x_Test.shape[0],1,42)
x_Test.shape

In [ ]:
import tensorflow.keras.layers as KL
inputs = KL.Input(shape=(1,42))
x = KL.LSTM(units = 8, activation = 'relu')(inputs)
outputs = KL.Dense(units=1, activation= 'sigmoid')(x)

model = tf.keras.models.Model(inputs, outputs)

opt = tf.keras.optimizers.Adamax()
model.compile( optimizer=opt , loss='binary_crossentropy', metrics = ['accuracy'])
history = model.fit(x_Train, y_Train, batch_size=20, epochs=1000, class_weight={0:1, 1:2},validation_data =(x_Test,y_Test))

#y_pred = model.predict(x_Test)
#print(y_pred)


In [ ]:
#from sklearn.metrics import confusion_matrix, classification_report,mean_squared_error
#confudf = pd.DataFrame(confusion_matrix(y_Test,y_pred), index = ["Negative (Actual)","Positive (Actual)"], columns=['Negative (Predicted)', 'Postive (Predicted)'])
#text = np.asarray([['True Negative', 'False Positive'], ['False Negative', 'True Positive']])
#labels = (np.asarray(["{0}\n{1:.2f}".format(text,data) for text, data in zip(text.flatten(), confusion_matrix(y_Test,y_pred).flatten())])).reshape(2,2)


#sns.heatmap(confudf, annot = labels, center=100, fmt='', linewidth = '1', vmax=300, cmap = "Blues", cbar=False)

#plt.yticks(rotation = 0)

In [ ]:
#model for Valence -LSTM

#x_Train = x_Train.reshape(x_Train.shape[0],1,1)
#x_Test = x_Test.reshape(x_Test.shape[0],1,1)

model = Sequential()
model.add(LSTM(128,activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation ='relu'))
model.add(Dropout(0.2))

model.add(Dense(32,activation ='relu'))
model.add(Dense(2,activation ='sigmoid'))
y_pred = model.predict(x_Test)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer =opt,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer =opt,metrics=['accuracy'])
history = model.fit(x_Train,y_Train,batch_size=16,epochs=200, validation_data =(x_Test,y_Test))
#print(y_pred)

In [ ]:
print(y_pred)

In [ ]:
model.summary()

In [ ]:
#Alternate model for Valence - CNN

#x_Train = x_Train.reshape(x_Train.shape[0],1,1)
#x_Test = x_Test.reshape(x_Test.shape[0],1,1)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(128,activation =tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(128,activation =tf.nn.relu))
model.add(Dense(10,activation =tf.nn.softmax))


opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)

model.compile(loss='sparse_categorical_crossentropy',optimizer =opt ,metrics=['accuracy'])
history_CNN = model.fit(x_Train,y_Train,epochs=100, validation_data =(x_Test,y_Test))

In [ ]:
model.summary()

In [ ]:
 plt.plot(history.history['loss'])
 plt.show()
 plt.plot(history_CNN.history['loss'])
 plt.show()


In [ ]:
#DATA PREP FOR AROUSAL

#Arousal is judged by alpha beta ratio of AF3 and AF4
#AF3 is data 1, AF4 is data 14
df3=data2.reset_index()['psdalpha_1']
df4=data2.reset_index()['psdbeta_1']

df34 = df4/df3

df5=data2.reset_index()['psdalpha_14']
df6=data2.reset_index()['psdbeta_14']

df56  = df5/df6

df7 = df34.append(df56)
df7

In [ ]:
target2 =data2['arousalhigh']
#target2 = target2.append(target2)
target2

In [ ]:
from sklearn.model_selection import train_test_split
#df7 = np.array(df7, dtype = float)
target2 = np.array(target2, dtype = float)
x_Train2, x_Test2, y_Train2, y_Test2 = train_test_split(dfz, target2, test_size = 0.35, random_state = 5)

In [ ]:
x_Train2 = x_Train2.reshape(x_Train2.shape[0],1,42)
x_Test2 = x_Test2.reshape(x_Test2.shape[0],1,42)
x_Test2.shape

In [ ]:
#model for Arousal

#x_Train2 = x_Train2.reshape(x_Train2.shape[0],1,1)
#x_Test2 = x_Test2.reshape(x_Test2.shape[0],1,1)

model = Sequential()
model.add(LSTM(128,activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation ='relu'))
model.add(Dense(32,activation ='relu'))
model.add(Dense(10,activation ='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer =opt,metrics=['accuracy'])
history = model.fit(x_Train2,y_Train2,epochs=1000, validation_data =(x_Test2,y_Test2))

In [ ]:
 plt.plot(history.history['loss'])
 plt.show()